In [ ]:
import collections

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import pamtra2

try:
    %matplotlib inline
except:
    pass


In [ ]:
wrfHydrometeors = ['CLOUD','RAIN','ICE','SNOW']#,'GRAUP']

wrfRenameDim = {
    'bottom_top_stag':'level',
    'bottom_top':'layer',
    'Time' : 'time',
}


def linLevel2layer(var):
    var= (var.isel(level=np.s_[:-1]) +  0.5 * var.diff('level'))
    return var.rename({'level':'layer'})


from pamtra2.libs.meteo_si.constants import Rair

In [ ]:
!ncdump -h /home/vagrant/shared/data/WRF/Barrow.nc

In [ ]:
wrfDat = xr.open_dataset('/home/vagrant/shared/data/WRF/Barrow.nc')
#fix time
wrfDat['Time'] = [np.datetime64(str(x).replace('_','T').replace("'",'').replace("b",''))
                          for x in wrfDat.Times.values]
wrfDat = wrfDat.drop('Times')
wrfDat = wrfDat.rename(wrfRenameDim)

wrfDat = wrfDat.squeeze()

In [ ]:
additionalDims = collections.OrderedDict()
additionalDims['time'] = wrfDat['time'].values
nHeights = len(wrfDat.layer)

pam2 = pamtra2.pamtra2(
    nLayer=nHeights,
    hydrometeors=wrfHydrometeors,
    additionalDims = additionalDims,
    frequencies = [35e9],
)




In [ ]:
pam2.profile

In [ ]:

heights = linLevel2layer((wrfDat["PH"]+wrfDat["PHB"])/9.81)
pressure = (wrfDat["P"] + wrfDat["PB"])


Cp = 7.*Rair/2.
RdCp =  Rair/Cp
p0 = 100000
T0 = wrfDat["T00"]
temperature = (wrfDat["T"] + T0)*(pressure/p0)**RdCp # potential temp to temp
temperature = temperature

horizontalWind = np.sqrt(wrfDat.V**2 + wrfDat.U**2)

mixingRatio = wrfDat.QVAPOR
specHum = mixingRatio/(1+mixingRatio)
relHum = pamtra2.libs.meteo_si.humidity.q2rh(specHum,temperature,pressure) * 100

pam2.profile.height[:] = heights
pam2.profile.temperature[:] = temperature
pam2.profile.relativeHumidity[:] = relHum # hardly relevant for radar simulator...
pam2.profile.pressure[:] = pressure
pam2.profile.eddyDissipationRate[:] = 1e-4 # not included in model output, 
pam2.profile.horizontalWind[:] =horizontalWind

pam2.addAirDensity()

pam2.profile['hydrometeorNumberConc'] = xr.zeros_like(pam2.profile.hydrometeorContent)

for hh, hydro in enumerate(wrfHydrometeors):
    
    pam2.profile.hydrometeorContent.values[...,hh] = wrfDat['Q%s'%hydro] * pam2.profile.airDensity
    if hydro not in ['CLOUD','GRAUP']:
        pam2.profile.hydrometeorNumberConc.values[...,hh] = wrfDat['QN%s'%hydro] * pam2.profile.airDensity
    elif hydro == 'CLOUD':
        cond = wrfDat['Q%s'%hydro]>0
        pam2.profile.hydrometeorNumberConc.values[cond,hh] = 2.5e8

pam2.addMissingVariables()


pam2.profile


In [ ]:

pam2.describeHydrometeor(
    pamtra2.hydrometeors.cloudDroplet,
    name = 'CLOUD',
    nBins =201,
    sizeDistribution = pamtra2.hydrometeors.sizeDistribution.exponentialN0WC, 
    scattering = pamtra2.hydrometeors.scattering.Mie,
    Dmin  = 1e-6,
    Dmax  = 4.04e-2,
    N0 =  8e6,
    useFuncArgDefaults = True,
)



In [ ]:
  data["PH"] = (data["PH"]+data["PHB"])/9.81 #now "real m
  del data["PHB"]

  data["P"] = data["P"]+data["PB"] # now in Pa
  del data["PB"]

  Cp = 7.*Rair/2.
  RdCp =  Rair/Cp
  p0 = 100000
  data["T"] = data["T"] + 300 # now in K
  data["T"] = data["T"]*(data["P"]/p0)**RdCp # potential temp to temp

  #add surface data
  data["TSK"] = data["TSK"].reshape(np.append(list(np.shape(data["TSK"])),1))
  data["PSFC"] = data["PSFC"].reshape(np.append(list(np.shape(data["PSFC"])),1))


In [ ]:
pamtra2.hydrometeors.cloudDroplet?

In [ ]:
wrfDat.QCLOUD


In [ ]:
mixingRatio = wrfDat.QVAPOR
specHum = mixingRatio/(1+mixingRatio)
relHum = pamtra2.libs.meteo_si.humidity.q2rh(specHum,temperature,pressure)



In [ ]:
temperature.plot()

In [ ]:
specHum.plot()

In [ ]:
pam2.profile.hydrometeorContent.sum('hydrometeor').plot()

In [ ]:
temperature.plot()